# Text Classifier

In [1]:
pip install transformers evaluate accelerate datasets scikit-learn torch tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 653.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 664.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 955.9 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 707.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 2.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 1.6 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━

In [23]:
pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 673.3 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 983.6 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=f22e3dfcdece9a557496250f2819a4692f4b52afd74908304e931bc23654bd6c
  Stored in directory: /Users/JoaoManuel/Library/Caches/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
train_df = pd.read_csv('ALLtraincompiled.csv')
test_df = pd.read_csv('ALLtestcompiled.csv')

train_df = train_df[['Question','Type']]
test_df = test_df[['Question','Type']]

# Split the dataset into training and test sets
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define the label mapping
label_mapping = {'counting_and_probability': 0, 'intermediate_algebra': 1, 'number_theory': 2, 'precalculus':3, 
                'prealgebra':4, 'geometry':5, 'algebra':6}
train_df['Type'] = train_df['Type'].map(label_mapping)
test_df['Type'] = test_df['Type'].map(label_mapping)

In [4]:
train_df['Type'].value_counts()

Type
6    1744
1    1295
4    1205
5     870
2     869
0     771
3     746
Name: count, dtype: int64

In [5]:
test_df['Type'].value_counts()

Type
6    1187
1     903
4     871
3     546
2     540
5     479
0     474
Name: count, dtype: int64

In [9]:
train_df.head(20)

,Question,Type
0,A board game spinner is divided into three par...,0
1,My school's math club has 6 boys and 8 girls. ...,0
2,How many 4-letter words with at least one cons...,0
3,Melinda will roll two standard six-sided dice ...,0
4,"Let $p$ be the probability that, in the proces...",0
5,How many four-digit numbers greater than 2999 ...,0
6,I have 5 marbles numbered 1 through 5 in a bag...,0
7,A box contains 5 white balls and 6 black balls...,0
8,The number of increasing sequences of positive...,0
9,Find the sum of all integers $k$ such that $\b...,0


In [16]:
import re

def remove_commas_from_latex_numbers(text):
    # Regular expression to match numbers with commas
    #pattern = r'(\$[0-9]\d{0,2}(,\!?\d{3})*(\.\d+)?\$)'
    pattern = r'(\$[0-9]\d{0,2}(,\\?\!?\\?\d{3})*(\.\d+)?\$)'

    # Function to remove commas from matched numbers
    def remove_commas(match):
        return match.group().replace(',', '').replace(r'\!', '')

    # Replace commas in matched numbers
    result = re.sub(pattern, remove_commas, text)
    return result

# Example input string
input_text = r"""
Cedric has deposited $\$12,\!000$ into an account that pays $5\%$ interest compounded annually.

Daniel has deposited $\$12,\!000$ into an account that pays $7\%$ simple annual interest.

In $15$ years Cedric and Daniel compare their respective balances. To the nearest dollar, what is the positive difference between their balances?
"""

# Apply the function to the input string
output_text = remove_commas_from_latex_numbers(input_text)

# Print the output string
print(output_text)



Cedric has deposited $\$12000$ into an account that pays $5\%$ interest compounded annually.

Daniel has deposited $\$12000$ into an account that pays $7\%$ simple annual interest.

In $15$ years Cedric and Daniel compare their respective balances. To the nearest dollar, what is the positive difference between their balances?



In [72]:
pip install pydetex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 2.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.5 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 973.4 kB/s eta 0:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.2/281.2 kB 1.1 MB/s eta 0:00:0000:0100:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=2997526b788a9cd6a4ab75e5dc26c6ec0c91e4a497ba7a2b091c3bb6cbe9462c
  Stored in directory: /Users/JoaoManuel/Library/Caches/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.w

In [125]:
import re

def handle_specific_commands(latex):

    command_pattern = r'\\([a-zA-Z]+)(?:\{(.*?)\})?(?:\{(.*?)\})?'
    
    # Define a dictionary to map LaTeX commands to their plain text equivalents
    command_mapping = {
        '\\frac': lambda args: f"{args[0]} divided by {args[1]}",
        '\\binom': lambda args: f"{args[0]} choose {args[1]}",
        '\\dbinom': lambda args: f"{args[0]} double choose {args[1]}"
        # Add more mappings as needed
    }
    
    # Function to handle LaTeX commands and their arguments
    def handle_command(match):
        command = match.group(1)
        arguments = match.group(2)[1:-1] if match.group(2) else ''  # Remove enclosing braces if present
        args = arguments.split('{')[1:] if arguments else []  # Split arguments if present
        args = [arg.split('}')[0] for arg in args]  # Remove closing braces
        if command in command_mapping:
            return command_mapping[command](args)
        else:
            return match.group(0)  # Return unchanged if command is not mapped
    
    # Replace LaTeX commands with their plain text equivalents
    plain_text = re.sub(command_pattern, handle_command, latex)
    
    return plain_text

In [175]:
import re
from pylatexenc.latex2text import LatexNodes2Text, MacroTextSpec

# class CustomLatexNodes2Text(LatexNodes2Text):
#     def __init__(self, latex_context=None):
#         if latex_context is None:
#             self.latex_context = self.get_default_latex_context_db()
#         super().__init__(latex_context=self.latex_context)

#     def get_custom_context(self):
#         context = self.latex_context
#         # Add custom macros here
#         context.add_context_category('custom-macros', [
#             MacroTextSpec('frac', args_parser='{}{}', macro_postproc=self.convert_frac),
#             MacroTextSpec('cdot', macro_postproc=lambda n, **kw: '·'),

#             # We can add more custom macros as needed but too many can lead to internal clashes with LatexNodes2Text internal translation
#         ])
#         return context

#     def convert_frac(self, n, **kwargs):
#         numerator = self.nodelist_to_text(n.nodeargd.argnlist[0])
#         denominator = self.nodelist_to_text(n.nodeargd.argnlist[1])
#         return f"({numerator} / {denominator})"

#LatexNodes2Text doesn't work for binomial, combinations, triple dots?
def custom_latex_to_text(latex):
    latex = re.sub(r'\\choose', ' choose', latex)
    latex = re.sub(r'\\binom\{(.*?)\}\{(.*?)\}', r'\1 choose \2', latex)
    latex = re.sub(r'\\dbinom\{(.*?)\}\{(.*?)\}', r'\1 choose \2', latex)
    latex = re.sub(r'\\cdots', '...', latex)
    latex = re.sub(r'\\cdot', '·', latex)                                    # multiplication
    latex = re.sub(r'\\spadesuit', 'spade' , latex)
    latex = re.sub(r'\\clubsuit', 'club', latex)
    latex = re.sub(r'\\diamondsuit', 'diamond' , latex)
    latex = re.sub(r'\\heartsuit', 'heart' , latex)
    latex = re.sub(r'\\frac\{(.*?)\}\{(.*?)\}', r'\1 / \2', latex)
    latex = re.sub(r'\\tfrac\{(.*?)\}\{(.*?)\}', r'\1 divided by \2', latex)
    latex = re.sub(r'\\lim\{(.*?)\}', r'lim as \1', latex)
    latex = re.sub(r'\\sin\{(.*?)\}', r'sin(\1)', latex)
    latex = re.sub(r'\\cos\{(.*?)\}', r'cos(\1)', latex)
    latex = re.sub(r'\\log\{(.*?)\}', r'log(\1)', latex)

    return latex

def remove_comma_from_latex(latex_list):
    # Regular expression to match numbers with commas
    #pattern = r'\$([1-9][0-9]{3,}|[1-9][0-9]{5,})'  
    pattern = r'(\$[0-9]\d{0,2}(,\\?\!?\\?\d{3})*(\.\d+)?\$)'
    
    # Function to remove commas from matched numbers
    def remove_comma(match):
        return match.group().replace(',', '')
    
    # Replace commas in numbers >= 1000 for each element in the list
    modified_latex_list = []
    for latex in latex_list:
        modified_latex = re.sub(pattern, remove_comma, latex)
        modified_latex_list.append(modified_latex)
        
    return modified_latex_list
# Test the function with an example
# latex_value = r"$12,\!000$"


# Function to extract LaTeX phrases
def extract_latex(text):
    pattern = r'\$(.*?)\$|\$\$(.*?)\$\$'
    expressions = re.findall(pattern, text, flags=re.DOTALL)
    return [expr[0] if expr[0] else expr[1] for expr in expressions]
    #pattern = r'\$[^\$]*?\$|\$\$[^\$]*?\$\$'
    #return re.findall(pattern, text)

# Function to convert LaTeX to plain text using both LatexNodes2Text and custom 
def convert_latex_to_text(latex_phrases):
    custom_converted = [custom_latex_to_text(phrase) for phrase in latex_phrases]
    # # Convert remaining LaTeX to plain text using LatexNodes2Text
    #custom_converted = [handle_specific_commands(phrase) for phrase in latex_phrases]
    converter = LatexNodes2Text()
    return [converter.latex_to_text(phrase) for phrase in custom_converted]


def replace_latex_with_text(question, latex_phrases, text_phrases):
    for latex, text in zip(latex_phrases, text_phrases):
        question = question.replace(f"${latex}$", text)
        #question = question.replace(latex, text)
    return question

# Extract LaTeX phrases and convert them
# train_df['Question'] = train_df['Question'].apply(remove_comma_from_latex)
train_df['Latex_Phrases'] = train_df['Question'].apply(extract_latex)
train_df['Math_Text'] = train_df['Latex_Phrases'].apply(convert_latex_to_text)
train_df['New_Question'] = train_df.apply(lambda row: replace_latex_with_text(row['Question'], row['Latex_Phrases'], row['Math_Text']), axis=1)


# # Print the DataFrame
#print(train_df[['Question','Latex_Phrases','Math_Text','New_Question']])
train_df.to_csv('questions_with_plain_text.csv', index=False)
train_df.to_excel('questions_with_plain_text.xlsx', index=False, engine='xlsxwriter')

train_df["Latex_Phrases"]
# train_df['Question']

0           [A, B, C, A, \frac{1}{3}, B, \frac{5}{12}, C]
1                                                      []
2                                [A, B, C, D, E, B, C, D]
3                                                      []
4                            [p, 5, 2, p, m/n, m, n, m+n]
                              ...                        
7495    [\displaystyle\frac{q}{r} = 9, \displaystyle\f...
7496                                       [x(x+2)+(x+2)]
7497    [\,  into an account that pays ,  interest com...
7498                          [\frac{1}{10}, \frac{1}{3}]
7499                                                   []
Name: Latex_Phrases, Length: 7500, dtype: object

In [159]:
train_df.head(50)["New_Question"]

0     A board game spinner is divided into three par...
1     My school's math club has 6 boys and 8 girls. ...
2     How many 4-letter words with at least one cons...
3     Melinda will roll two standard six-sided dice ...
4     Let p be the probability that, in the process ...
5     How many four-digit numbers greater than 2999 ...
6     I have 5 marbles numbered 1 through 5 in a bag...
7     A box contains 5 white balls and 6 black balls...
8     The number of increasing sequences of positive...
9     Find the sum of all integers k such that 23 ch...
10    Find the number of ordered pairs of positive i...
11    Determine the number of ways to arrange the le...
12    A mathematical organization is producing a set...
13    If I choose four cards from a standard 52-card...
14    In how many ways can I choose 3 captains from ...
15    A point P is randomly selected from the square...
16    Let P(x) = x^2 - 3x - 9. A real number x is ch...
17    Six distinct integers are picked at random

In [167]:
train_df["Question"].iloc[157]

"Will we have questions with tables?"


'In the game Deal or No Deal, participants choose a box at random from a set of $26,$ one containing each of the following values: \\begin{tabular}{|c|c|}\\hline\\$.01&\\$1,000\\\\\\hline\\$1&\\$5,000\\\\\\hline\\$5&\\$10,000\\\\\\hline\\$10&\\$25,000\\\\\\hline\\$25&\\$50,000\\\\\\hline\\$50&\\$75,000\\\\\\hline\\$75&\\$100,000\\\\\\hline\\$100&\\$200,000\\\\\\hline\\$200&\\$300,000\\\\\\hline\\$300&\\$400,000\\\\\\hline\\$400&\\$500,000\\\\\\hline\\$500&\\$750,000\\\\\\hline\\$750&\\$1,000,000\\\\\\hline\\end{tabular} After choosing a box, participants eliminate other boxes by opening them, showing the amount of money in the box to the crowd, and then removing that box (and its money!) from the game. What is the minimum number of boxes a participant needs to eliminate in order to have a half chance of holding at least $\\$100,\\!000$ as his or her chosen box?'

In [ ]:
train_df["Question"].iloc[203]

"Will we have questions with a draw functionality?"

In [176]:
train_df['New_Question'].iloc[353]

'A standard six-sided fair die is rolled four times. The probability that the product of all four numbers rolled is a perfect square is m divided by n, where m and n are relatively prime positive integers. Find m+n.\n'

In [161]:
train_df['New_Question'].iloc[109]

'Simplify \\frac{11!}{9! + 2\\cdot 8!}'

In [163]:
train_df["New_Question"].iloc[80]

'Find the last two digits of the following sum: 5! + 10! + 15! + \\cdots + 100!'

In [162]:
train_df['Latex_Phrases'].iloc[80]

['', '']

In [165]:
train_df['New_Question'].iloc[37]

'A standard deck of 52 cards has 13 ranks (Ace, 2, 3, 4, 5, 6, 7, 8, 9, 10, Jack, Queen, King) and 4 suits (spade, heart, diamond, and club), such that there is exactly one card for any given rank and suit.  Two of the suits (spade and club) are black and the other two suits (heart and diamond) are red.  The deck is randomly arranged. What is the probability that the top three cards are all spades?'

https://github.com/purdue-tlt/latex2sympy/tree/master

Use this in terminal to download

pip install wheel
pip install "cython<3.0.0" && pip install --no-build-isolation pyyaml==6.0 


In [1]:
pip install latex2sympy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 5.1 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.7.2-py3-none-any.whl size=140931 sha256=929bede0eda7eb678ffa16b4431046a1067e1fe97d7dae34f879106297706035
  Stored in directory: /Users/jameskim/Library/Caches/pip/wheels/24/09/d5/906109a0b239813cee4d939fa6ccee45f63a19e3f774405e34
Successfully built antlr4-python3-runtime
Note: you may need to restart the kernel to use updated packages.


In [49]:
latexx= '$\$12,\!000$'
latex2sympy(latexx)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/var/folders/bs/zggx74yj4d721rnp57yp6h5r0000gn/T/ipykernel_52452/721610512.py:1: SyntaxWarning: invalid escape sequence '\$'
  latexx= '$\$12,\!000$'


Exception: I don't understand this
 \ 12,\!000 
~^

In [6]:
from latex2sympy2 import latex2sympy

train_df['sympy_column'] = train_df['latex_column'].apply(convert_to_sympy)
print(x)
# => "diff(x**(2), x)"

binomial(m, n)


<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
/var/folders/bs/zggx74yj4d721rnp57yp6h5r0000gn/T/ipykernel_52452/3262760491.py:3: SyntaxWarning: invalid escape sequence '\c'
  x = latex2sympy("${m \choose n}$")


In [33]:
from datasets import Dataset
from transformers import BertTokenizer

# Convert DataFrames to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', add_special_tokens=True)


latex_pattern = r'\$([^$]*)\$'

# Tokenization function with LaTeX support
def tokenize_function(examples):
    # Tokenize each problem while preserving LaTeX
    problem_text = examples['Question']
    # Find LaTeX expressions and tokenize them separately
    matches = re.finditer(latex_pattern, problem_text)
    tokenized_problem = []
    last_end = 0
    for match in matches:
        start, end = match.span()
        # Tokenize text before LaTeX expression
        tokenized_problem.extend(tokenizer.tokenize(problem_text[last_end:start]))
        # Tokenize LaTeX expression and add each token separately
        latex_expression = match.group(1)
        tokenized_latex = tokenizer.tokenize(latex_expression)
        tokenized_problem.extend(tokenized_latex)
        last_end = end
    # Tokenize the remaining text after the last LaTeX expression
    tokenized_problem.extend(tokenizer.tokenize(problem_text[last_end:]))
    return tokenizer.pad({'input_ids': tokenized_problem}, return_tensors='pt')

# Tokenization function
# Tokenization function with debugging
# def tokenize_function(examples):
#     tokenized_inputs = tokenizer(examples['Question'], padding='max_length', truncation=True)
#     # print("Example:", examples['Question'])
#     # print("Tokenized inputs:", tokenized_inputs)
#     return tokenized_inputs


# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


# Set format for PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Type'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Type'])


Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

TypeError: expected string or bytes-like object, got 'list'

In [14]:
pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.6 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
import torch
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# Load BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Apply LoRA to the BERT model
config = LoraConfig(
    r=8,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor for the LoRA updates
    target_modules=["query", "key", "value"],  # Apply LoRA to these layers
    lora_dropout=0.1,  # Dropout rate
)

model = get_peft_model(model, config)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/homebrew/Cellar/jupyterlab/4.2.0/libexec/lib/python3.12/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


IndexError: Invalid key: 7278 is out of bounds for size 0